## **Imports**


In [16]:
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import argparse
#import imutils  # If you are unable to install this library, ask the TA; we only need this in extract_hsv_histogram.
import cv2
import os
import re
import random
import pytesseract
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from skimage.util import random_noise
from PIL import Image, ImageDraw, ImageFont
from arabic_reshaper import reshape
from bidi.algorithm import get_display
import pandas as pd
from openpyxl.utils import get_column_letter  # Add this line
from commonfunctions import *
import numpy as np
import unittest

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

import cv2
import numpy as np
import matplotlib.pyplot as plt

pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe' # (Windows Example)


target_img_size = (32, 32) # fix image size because classification algorithms THAT WE WILL USE HERE expect that
# We are going to fix the random seed to make our experiments reproducible 
# since some algorithms use pseudorandom generators
random_seed = 42  
random.seed(random_seed)
np.random.seed(random_seed)

In [17]:
# # === Single-ID preprocessing debug (set RAW_ID and run) ===
# import os
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# from PIL import Image

# RAW_ID = 7  # <- change this (e.g., 2, 14, 37...)
# REFERENCE_ID = 14  # reference card used for alignment
# EXT = '.jpg'
# APPLY_GLARE_REMOVAL = True  # <- set False to skip
# GLARE_V_THRESH = 235
# GLARE_S_THRESH = 80
# APPLY_PIL_UPSCALE = True  # <- set False to load with cv2.imread only


# def enhance_image_quality(image_path):
#     """Enhance image quality to improve OCR accuracy."""
#     try:
#         img = Image.open(image_path)
#         # Increase resolution if image is small
#         if max(img.size) < 800:
#             new_size = (img.size[0] * 2, img.size[1] * 2)
#             img = img.resize(new_size, Image.Resampling.LANCZOS)
#         return img
#     except Exception:
#         return Image.open(image_path)


# def load_image_bgr(image_path: str) -> np.ndarray:
#     """Load image as OpenCV BGR, optionally upscaling with PIL first."""
#     if APPLY_PIL_UPSCALE:
#         try:
#             pil_img = enhance_image_quality(image_path).convert('RGB')
#             rgb = np.array(pil_img)
#             return cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)
#         except Exception:
#             pass

#     return cv2.imread(image_path)


# def _show(img, title):
#     if img is None:
#         print(f"{title}: <None>")
#         return
#     plt.figure(figsize=(6, 4))
#     if len(img.shape) == 2:
#         plt.imshow(img, cmap='gray')
#     else:
#         plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#     plt.title(title)
#     plt.axis('off')
#     plt.show()


# base_dir = os.getcwd()
# raw_path = os.path.join(base_dir, 'Raw_IDs', f'ID{RAW_ID}{EXT}')
# ref_path = os.path.join(base_dir, 'Raw_IDs', f'ID{REFERENCE_ID}{EXT}')

# if not os.path.exists(raw_path):
#     raise FileNotFoundError(f"Raw image not found: {raw_path}")
# if not os.path.exists(ref_path):
#     raise FileNotFoundError(f"Reference image not found: {ref_path}")

# raw_img = load_image_bgr(raw_path)
# _show(raw_img, f"Step 0 - Raw (ID{RAW_ID})")

# if APPLY_GLARE_REMOVAL:
#     glare_fixed, glare_mask = remove_glare_spots(raw_img, v_thresh=GLARE_V_THRESH, s_thresh=GLARE_S_THRESH)
#     _show(glare_mask, "Step 0b - Glare mask")
#     _show(glare_fixed, "Step 0c - After glare removal")
#     img0 = glare_fixed
# else:
#     img0 = raw_img

# clean_img, is_impulsive = is_impulsive_noise(img0)
# print('Impulsive noise detected:', is_impulsive)
# _show(clean_img, "Step 1 - After impulsive noise removal")

# aligned_img = align_images_sift(clean_img, ref_path)
# _show(aligned_img, f"Step 2 - After alignment to ID{REFERENCE_ID}")

# clean_img2, is_random = is_random_noise(aligned_img)
# print('Random noise detected:', is_random)
# _show(clean_img2, "Step 3 - After random noise removal")

# enhanced_img = enhance_contrast_clahe(clean_img2)
# _show(enhanced_img, "Step 4 - After contrast enhancement")

# name_img, digit_imgs, daf3_digits = extract_name_and_digits(enhanced_img)
# _show(name_img, "Step 5a - Cropped name region")

# # Show code digits (first row)
# if isinstance(digit_imgs, (list, tuple)) and len(digit_imgs) > 0:
#     for idx, d in enumerate(digit_imgs):
#         _show(d, f"Step 5b - Code digit {idx}")
# else:
#     print('No code digits returned from extract_name_and_digits')

# # Show Daf3 digits
# if isinstance(daf3_digits, (list, tuple)) and len(daf3_digits) > 0:
#     for idx, d in enumerate(daf3_digits):
#         _show(d, f"Step 5c - Daf3 digit {idx}")
# else:
#     print('No Daf3 digits returned from extract_name_and_digits')

## GPU setup (EasyOCR / Qwen / PaddleOCR)
Run the next cell to check if CUDA is available in this environment.

In [18]:
import os
import platform
import sys

print('OS:', platform.platform())
print('Python:', sys.version)
print('Executable:', sys.executable)

# Workaround for a common Windows conda issue (harmless if not needed)
# If you see: "OMP: Error #15: Initializing libiomp5md.dll..." then keep this.
os.environ.setdefault('KMP_DUPLICATE_LIB_OK', 'TRUE')

# ---- NumPy ----
try:
    import numpy as np
    print('numpy:', np.__version__)
except Exception as e:
    print('numpy import failed:', e)

# ---- PyTorch (EasyOCR + Qwen) ----
try:
    import torch
    print('torch:', torch.__version__)
    print('torch cuda_available:', torch.cuda.is_available())
    if torch.cuda.is_available():
        print('torch cuda version:', torch.version.cuda)
        print('torch device 0:', torch.cuda.get_device_name(0))
except Exception as e:
    print('torch import failed:', e)

# ---- PaddlePaddle (PaddleOCR) ----
try:
    import paddle
    print('paddle:', paddle.__version__)
    print('paddle cuda_compiled:', paddle.device.is_compiled_with_cuda())
except Exception as e:
    print('paddle import failed:', e)

OS: Windows-11-10.0.26200-SP0
Python: 3.12.6 (tags/v3.12.6:a4a2d2b, Sep  6 2024, 20:11:23) [MSC v.1940 64 bit (AMD64)]
Executable: c:\Program Files\Python312\python.exe
numpy: 2.2.6
torch: 2.9.1+cpu
torch cuda_available: False
paddle import failed: No module named 'paddle'


### What to do to run on GPU (and fix EasyOCR import errors)
1) Run the GPU check cell above.
- If it prints `torch cuda_available: True` then EasyOCR + Qwen will use GPU.
- If it prints `torch ... +cpu` or `torch cuda_available: False`, install CUDA PyTorch (recommended in conda):
  - `conda install -y -n imageproc pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia`
  - Restart the notebook kernel and re-run the GPU check.

2) If you get this error when importing EasyOCR / torchvision:
`ImportError: cannot import name 'DiagnosticOptions' from 'torch.onnx._internal.exporter'`
That means your environment has **mismatched** `torch` and `torchvision` (usually because of pip installs on top of conda). Fix by removing pip torch packages and reinstalling the conda stack:
- `python -m pip uninstall -y torch torchvision torchaudio`
- `conda install -y -n imageproc pytorch torchvision torchaudio pytorch-cuda=12.1 -c pytorch -c nvidia`
- Restart the notebook kernel.

3) PaddleOCR GPU note (optional)
- Your current pipeline uses **EasyOCR** for names (not PaddleOCR), so Paddle CUDA is **not required** unless you switch back to PaddleOCR.

In [19]:
# Quick verification (run after you restart the kernel)
import os
os.environ.setdefault('KMP_DUPLICATE_LIB_OK', 'TRUE')

import numpy as np
import torch
import torchvision
import easyocr

print('numpy:', np.__version__)
print('torch:', torch.__version__, 'cuda_available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
print('torchvision:', torchvision.__version__)
print('easyocr:', easyocr.__version__)

numpy: 2.2.6
torch: 2.9.1+cpu cuda_available: False
torchvision: 0.24.1+cpu
easyocr: 1.7.2


## **Main Functions Overview**

- **Image Alignment**
  - Detects SIFT keypoints and descriptors in the input and reference images.
  - Matches them using the ratio test.
  - Uses RANSAC to estimate a homography.
  - Applies the homography to warp the input image so it lines up with the reference.
  - Returns the aligned image (or the original if not enough matches are found).

- **Extract Details**
  - Uses (x, y, w, h) coordinates to crop the aligned card into:
    - The name region
    - The code (ID) region
  - Returns these sub-images for downstream OCR or digit processing.

- **Save Student Name**
  - Ensures the output folder exists.
  - Writes the cropped name image to disk with a filename that includes the student ID.
  - Creates a persistent record usable for manual review or OCR.

- **Split and Save Digits**
  - Converts the code region to grayscale and applies Otsu thresholding.
  - Finds contours and filters out small noise.
  - Selects the largest seven contours (by area) and sorts them left-to-right.
  - Saves each detected digit crop into a per-student folder as individual image files.

- **save_split_digits**
  - Takes a list of digit images for a student.
  - Ensures a folder exists for each student (named by their ID).
  - Saves each digit image as `digit_0.jpg`, `digit_1.jpg`, ..., `digit_6.jpg` inside the student’s folder.
  - Used for batch saving when all digit crops are already extracted.

## **Noise Detection and Treatment**
- **Impulsive Noise (Median Filter)**
- **Random Noise (Gaussian Filter)**

In [20]:
import cv2
import numpy as np

def is_random_noise(img, threshold=0.1):
    """
    Detects random noise and applies Non-Local Means (NLM) Denoising.
    This is the best traditional filter for preserving textures and sharp edges.
    """
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Estimate noise level
    stddev = np.std(img)
    normalized_stddev = stddev / 255.0

    if normalized_stddev < threshold:
        return img, False

    # cv2.fastNlMeansDenoising(src, h, templateWindowSize, searchWindowSize)
    # h: Parameter deciding filter strength. Higher = more noise removal but less detail. 
    #    (10 is usually a good starting point for moderate noise)
    # templateWindowSize: Size of the patches used to compute weights (typically 7).
    # searchWindowSize: Size of the window to search for similar patches (typically 21).
    
    treated_img = cv2.fastNlMeansDenoising(
        img, 
        None, 
        h=10, 
        templateWindowSize=7, 
        searchWindowSize=21
    )
    
    return treated_img, True

In [21]:
import cv2
import numpy as np

def is_impulsive_noise(img, threshold=0.1, black_range=(0, 9), white_range=(246, 255)):
    """
    Detects impulsive noise and applies a Decision-Based Median Filter.
    Only noisy pixels are modified; clean pixels remain sharp and unblurred.
    """
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
    total_pixels = img.size
    # Create masks for pixels that fall into the 'salt' or 'pepper' ranges
    is_pepper = (img >= black_range[0]) & (img <= black_range[1])
    is_salt = (img >= white_range[0]) & (img <= white_range[1])
    
    # Combined noise mask
    noise_mask = is_pepper | is_salt
    num_noise_pixels = np.sum(noise_mask)
    prop = num_noise_pixels / total_pixels

    if prop < threshold:
        return img, False 

    # Determine kernel size based on noise severity
    k = int(3 + prop * 10)
    if k % 2 == 0: k += 1
    k = min(max(k, 3), 9)

    # 1. Apply a standard median blur to a temporary image
    median_filtered = cv2.medianBlur(img, k)

    # 2. DECISION STEP: Create output as a copy of the original
    # This ensures we don't blur the whole image.
    treated_img = img.copy()

    # 3. Only replace pixels where the noise_mask is True
    treated_img[noise_mask] = median_filtered[noise_mask]

    return treated_img, True

Contrast enhancment

In [22]:
import cv2
import numpy as np

def enhance_contrast_clahe(img, clip_limit=2.0, tile_size=(8, 8)):
    """
    Applies CLAHE for high-quality contrast enhancement.
    Returns the enhanced image.
    """
    if len(img.shape) == 3:
        # Convert to LAB to enhance only the Luminance (L) channel
        # This prevents color distortion.
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        
        # Create CLAHE object
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_size)
        cl = clahe.apply(l)
        
        # Merge back and convert to BGR
        enhanced_lab = cv2.merge((cl, a, b))
        return cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)
    else:
        # Grayscale implementation
        clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_size)
        return clahe.apply(img)

In [23]:
def boost_black_text(img):
    # Handle both color and grayscale images
    if len(img.shape) == 2:
        # Grayscale input: apply CLAHE directly
        clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8,8))
        return clahe.apply(img)
    else:
        # Color input: convert to LAB, enhance L channel, convert back
        lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab)
        clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8,8))
        l = clahe.apply(l)
        lab = cv2.merge((l,a,b))
        return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)


## Flashlight Spot

In [24]:
import cv2
import numpy as np

def remove_glare_spots(
    img,
    v_thresh=225,
    s_thresh=85,        
    dilate_iter=2,
    inpaint_radius=4
):
    if img is None:
        return None, None

    orig_dtype = img.dtype

    # --- Ensure uint8 ---
    img_u8 = img
    if img_u8.dtype != np.uint8:
        img_f = img_u8.astype(np.float32)
        if img_f.size and float(np.nanmax(img_f)) <= 1.0:
            img_u8 = np.clip(img_f * 255.0, 0, 255).astype(np.uint8)
        else:
            img_u8 = np.clip(img_f, 0, 255).astype(np.uint8)

    was_gray = (img_u8.ndim == 2)
    if was_gray:
        bgr = cv2.cvtColor(img_u8, cv2.COLOR_GRAY2BGR)
    else:
        if img_u8.shape[2] == 4:
            bgr = cv2.cvtColor(img_u8, cv2.COLOR_BGRA2BGR)
        else:
            bgr = img_u8

    # --- Glare detection ---
    hsv = cv2.cvtColor(bgr, cv2.COLOR_BGR2HSV)
    s = hsv[:, :, 1]
    v = hsv[:, :, 2]

    mask = ((v >= v_thresh) & (v >= 180) & (s <= s_thresh)).astype(np.uint8) * 255

    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)

    if dilate_iter > 0:
        mask = cv2.dilate(mask, kernel, iterations=dilate_iter)

    # --- PROTECT DARK DIGITS ---
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
    DIGIT_BLACK_THRESH = 140
    mask[gray < DIGIT_BLACK_THRESH] = 0

    # Shrink mask so neighbors stay untouched
    shrink_kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    safe_mask = cv2.erode(mask, shrink_kernel, iterations=1)

    if safe_mask.max() == 0:
        return img.copy(), safe_mask

    # --- Inpainting ---
    inpainted = cv2.inpaint(bgr, safe_mask, inpaint_radius, cv2.INPAINT_TELEA)

    # Soft blend
    alpha = cv2.GaussianBlur(safe_mask.astype(np.float32) / 255.0, (0, 0), 2.0)
    alpha3 = alpha[:, :, None]
    blended = (bgr.astype(np.float32) * (1 - alpha3) +
               inpainted.astype(np.float32) * alpha3)
    blended = np.clip(blended, 0, 255).astype(np.uint8)

    out_u8 = cv2.cvtColor(blended, cv2.COLOR_BGR2GRAY) if was_gray else blended

    # --- Restore dtype ---
    if orig_dtype == np.uint8:
        out = out_u8
    elif np.issubdtype(orig_dtype, np.floating):
        if float(np.nanmax(img)) <= 1.0:
            out = (out_u8.astype(np.float32) / 255.0).astype(orig_dtype)
        else:
            out = out_u8.astype(orig_dtype)
    else:
        out = out_u8.astype(orig_dtype)

    return out, safe_mask


In [25]:
def align_images_sift(img_to_align, reference_path):
    img1 = img_to_align
    img2 = cv2.imread(reference_path)      # Train Image (The perfect template)
    
    # --- FIX: Check if img1 is already grayscale ---
    if len(img1.shape) == 3:
        gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    else:
        gray1 = img1 # Already grayscale

    # Ref image is loaded from disk, usually BGR, but good to check
    if len(img2.shape) == 3:
        gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    else:
        gray2 = img2

    sift = cv2.SIFT_create() 
    
    kp1, des1 = sift.detectAndCompute(gray1, None)
    kp2, des2 = sift.detectAndCompute(gray2, None)

    bf = cv2.BFMatcher()
    matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    if len(good_matches) > 10:
        src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

        M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

        h, w = img2.shape[:2]
        # Warp the original img1 (whether gray or color)
        aligned_img = cv2.warpPerspective(img1, M, (w, h))

        return aligned_img
    
    else:
        print(f"Not enough matches found: {len(good_matches)}/10")
        return img1
    


def extract_details(aligned_image):
    name_coords = (100, 205, 1200, 150)
    code_coords = (640, 404, 335, 110)
    
    nx, ny, nw, nh = name_coords
    cx, cy, cw, ch = code_coords
    
    name_contour = aligned_image[ny:ny+nh, nx:nx+nw]
    code_contour = aligned_image[cy:cy+ch, cx:cx+cw]
    
    return name_contour, code_contour


def save_student_name(student_id, name_img, output_folder="extracted_names"):
    # Create folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    # Construct filename: extracted_names/ID1_name.jpg
    filename = f"{output_folder}/{student_id}_name.jpg"
    
    # Save the image
    cv2.imwrite(filename, name_img)
    

def split_and_save_digits(student_id, code_roi, output_folder="extracted_digits"):
    save_path = f"{output_folder}/ID{student_id}"
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        
    gray = cv2.cvtColor(code_roi, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # A. Collect all valid candidates
    candidates = []
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        area = w * h
        # Filter tiny noise
        if h > 15 and w > 5:
            candidates.append((x, y, w, h, area))
            
    # B. CRITICAL: Select exactly the top 7 by AREA (Size)
    # This removes small specks or the colon ":" if it was caught
    candidates = sorted(candidates, key=lambda c: c[4], reverse=True) # Sort largest first
    final_digits = candidates[:7] # Take top 7
    
    # C. Sort the final 7 by X-COORDINATE (Left -> Right)
    # This puts them back in the correct reading order (1, 2, 3...)
    final_digits = sorted(final_digits, key=lambda c: c[0])
        
    # D. Save
    for index, (x, y, w, h, area) in enumerate(final_digits):
        digit_img = code_roi[y:y+h, x:x+w]
        filename = f"{save_path}/digit_{index}.jpg"
        cv2.imwrite(filename, digit_img)

      
import cv2
import numpy as np

def extract_name_and_digits(aligned_image):
    """
    Input: An aligned ID card image.
    Output: 
      - name_roi: The image of the extracted name.
      - code_digits: A list of images for the code digits.
      - daf3_digits: A list of images for the daf3 digits.
    """
    
    # --- 1. Hardcoded Coordinates ---
    # x, y, w, h
    name_coords = (100, 205, 1200, 150)
    code_coords = (640, 404, 335, 110)
    daf3_coords = (350, 500, 620, 110)
    
    nx, ny, nw, nh = name_coords
    cx, cy, cw, ch = code_coords
    dx, dy, dw, dh = daf3_coords
    
    # Extract ROIs
    name_img = aligned_image[ny:ny+nh, nx:nx+nw]
    code_roi = aligned_image[cy:cy+ch, cx:cx+cw]
    daf3_img = aligned_image[dy:dy+dh, dx:dx+dw]
    
    # --- Helper Function to Process Any ROI ---
    def process_roi_digits(roi_img, digit_limit):
        """
        Applies grayscale, thresholding, contour detection, 
        splitting of merged digits, and sorting.
        """
        #gray = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)

        if len(roi_img.shape) == 3:
            gray = cv2.cvtColor(roi_img, cv2.COLOR_BGR2GRAY)
        else:
            gray = roi_img
        # --- FIX ENDS HERE --
        
        # Binary Inverse + Otsu
        _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        candidates = []
        for cnt in contours:
            x, y, w, h = cv2.boundingRect(cnt)
            area = w * h
            
            # Filter tiny noise 
            if h > 15 and w > 5:
                
                # --- CHECK FOR MERGED DIGITS ---
                # If width > 0.8 * height, likely two digits stuck together
                if w > 0.8 * h: 
                    half_w = w // 2
                    # Digit 1 (Left half)
                    candidates.append((x, y, half_w, h, half_w * h))
                    # Digit 2 (Right half)
                    candidates.append((x + half_w, y, half_w, h, half_w * h))
                else:
                    # Normal single digit
                    candidates.append((x, y, w, h, area))
        
        # 1. Sort by Area Descending (Keep only the largest objects to remove noise)
        candidates = sorted(candidates, key=lambda c: c[4], reverse=True)[:digit_limit]
        
        # 2. Sort by X-coordinate Ascending (Order them Left -> Right)
        final_candidates = sorted(candidates, key=lambda c: c[0])
        
        # Crop the actual images
        cropped_digits = []
        for (x, y, w, h, area) in final_candidates:
            digit_crop = roi_img[y:y+h, x:x+w]
            cropped_digits.append(digit_crop)
            
        return cropped_digits

    # --- 2. Process Regions ---
    
    # Detect Code (Limit 7 digits)
    code_digits = process_roi_digits(code_roi, digit_limit=7)
    
    # Detect Daf3 (Limit 14 digits)
    daf3_digits = process_roi_digits(daf3_img, digit_limit=14)

    return name_img, code_digits, daf3_digits

def save_split_digits(student_id, digit_imgs, output_folder="extracted_digits"):
    """
    Saves a list of digit images for a student in the same way as split_and_save_digits.
    Each digit is saved as digit_0.jpg, digit_1.jpg, ..., digit_6.jpg in a folder per student.
    """
    save_path = f"{output_folder}/{student_id}"
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for index, digit_img in enumerate(digit_imgs):
        filename = f"{save_path}/digit_{index}.jpg"
        cv2.imwrite(filename, digit_img)    



## **SVM English Number Classifier**

In [26]:
path_to_train_dataset = r"train_digits"  # Training set

def train_SVM_robust():
    # 1. Map your specific filename prefixes to actual digits
    label_map = {
        'a': '0', 'b': '1', 'c': '2', 'd': '3', 'e': '4', 
        'f': '5', 'g': '6', 'h': '7', 'i': '8', 'j': '9'
    }
    
    features = []
    labels = []
    
    img_filenames = os.listdir(path_to_train_dataset)
    print(f"Loading {len(img_filenames)} training images...")

    for fn in img_filenames:
        if not fn.lower().endswith(('.jpg', '.png')):
            continue

        # Get the first letter (a, b, c...)
        prefix = fn[0].lower()
        if prefix in label_map:
            labels.append(label_map[prefix])
            
            path = os.path.join(path_to_train_dataset, fn)
            img = cv2.imread(path)
            
            # Extract HOG features (ensure preprocessing matches)
            features.append(extract_hog_features(img))
    
    # 2. Create a Pipeline: Scale Features -> Train SVM
    # Scaling is CRITICAL for HOG-based SVMs
    clf = Pipeline([
        ('scaler', StandardScaler()),
        ('svc', LinearSVC(random_state=42, max_iter=5000, dual=False))
    ])
    
    # 3. Train/Test Split for internal validation
    X_train, X_test, y_train, y_test = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed
    )
    
    clf.fit(X_train, y_train)
    accuracy = clf.score(X_test, y_test)
    print(f"Training Complete. Validation Accuracy: {accuracy*100:.2f}%")
    
    return clf

def extract_hog_features(img):
    # Ensure grayscale
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # APPLY IDENTICAL PREPROCESSING TO TRAINING AND TEST DATA
    # This turns both sets into "binary masks" to ignore lighting/shadows
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    img = cv2.resize(img, (32, 32)) # target_img_size
    
    win_size = (32, 32)
    cell_size = (8, 8)  # Slightly larger cells help ignore "noise/shadows"
    block_size = (16, 16)
    block_stride = (8, 8)
    nbins = 9
    
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    return h.flatten()

# **Tesseract Arabic OCR**

## **Current Situation**

The project uses Tesseract OCR to extract Arabic names from scanned images. Initially, the extraction pipeline achieved only a **70% success rate**. This meant that about 30% of the images failed to yield any valid Arabic text, even though the images were visually clear and contained readable names.

## **Why Was the Success Rate Only 70%?**

- **Overprocessing:** The original code applied several preprocessing steps (scaling, thresholding, blurring, etc.) before running OCR. While these steps can help with noisy or low-contrast images, they often **destroy clean, high-contrast text**—especially for Arabic, where fine details matter.
- **Order of Operations:** The pipeline tried processed versions first, so if the original image was already optimal, it was never used for OCR.
- **PSM/OEM Settings:** The code tried a limited set of Tesseract Page Segmentation Modes (PSM) and OCR Engine Modes (OEM), which may not have been optimal for all images.
- **Text Cleaning:** The cleaning function was aggressive, but if Tesseract output was empty or too short, the result was discarded.

## **What Was Changed to Achieve 100% Success**

1. **Prioritize the Original Image:**  
   The new code always tries the original, unprocessed grayscale image first, with several PSM settings. This ensures that clean images are not degraded by unnecessary processing.

2. **Expanded Preprocessing (But Only If Needed):**  
   Only if the original image fails, the code tries padded and scaled versions, but never applies destructive thresholding or blurring unless absolutely necessary.

3. **Multiple PSM and OEM Combinations:**  
   For each image variant, the code tries several PSM (6, 7, 3, 13) and both OEM (3, 1) settings, maximizing the chance that Tesseract will interpret the layout correctly.

4. **Result Selection:**  
   All non-empty results are collected, and the **longest valid extraction** is chosen, which is usually the correct full name.

5. **Diagnostics:**  
   Additional debug and diagnostic code was used to confirm that the original image, with minimal processing, consistently yields the best results for this dataset.

# Reference

The old (70%) code is left in the notebook for comparison. The new approach, as described above, achieves **100% extraction success** on the current dataset by respecting the quality of the input images and leveraging Tesseract's flexibility.

In [27]:
# def extractname(img_path):
    
#     # --- HELPER: TEXT CLEANER ---
#     def clean_text(raw_text):
#         if not raw_text: return ""
#         # Keep Arabic letters (0621-064A) and spaces
#         cleaned = re.sub(r'[^\u0621-\u064A\s]', '', raw_text)
#         cleaned = cleaned.replace('\n', ' ')
#         cleaned = re.sub(r'\s+', ' ', cleaned).strip()
#         return cleaned

#     # --- LOAD IMAGE AS GRAYSCALE DIRECTLY ---
#     img_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
#     if img_gray is None: 
#         return ""

#     # Try multiple approaches and collect all results
#     all_results = []
    
#     # Preprocessing variants
#     preprocessed_images = {
#         'original': img_gray,
#         'padded': cv2.copyMakeBorder(img_gray, 40, 40, 40, 40, cv2.BORDER_CONSTANT, value=255),
#     }
    
#     # Add scaled version
#     h, w = img_gray.shape
#     scaled = cv2.resize(img_gray, (w*2, h*2), interpolation=cv2.INTER_CUBIC)
#     preprocessed_images['scaled_padded'] = cv2.copyMakeBorder(scaled, 40, 40, 40, 40, cv2.BORDER_CONSTANT, value=255)
    
#     # PSM modes to try
#     psm_modes = [6, 7, 3, 13]  # 13 = raw line
    
#     for img_name, img in preprocessed_images.items():
#         for psm in psm_modes:
#             for oem in [3, 1]:  # Try both LSTM+Legacy and LSTM only
#                 try:
#                     config = f"--oem {oem} --psm {psm}"
#                     text = pytesseract.image_to_string(img, lang='ara', config=config)
#                     cleaned = clean_text(text)
                    
#                     if len(cleaned) > 2:
#                         all_results.append((cleaned, len(cleaned), img_name, psm, oem))
#                 except:
#                     continue
    
#     # Return the longest valid result
#     if all_results:
#         all_results.sort(key=lambda x: x[1], reverse=True)
#         return all_results[0][0]
    
#     return ""








###############################    easyOCR       #####################################



import easyocr
reader = easyocr.Reader(['ar', 'en'], gpu=True)

def extractname(image_path):
    """
    Reads text from the image path using EasyOCR.
    """
    try:
        results = reader.readtext(image_path, detail=0, paragraph=True)
        
        # Join all detected text parts into one string
        full_name = " ".join(results)
        return full_name.strip()
    except Exception as e:
        print(f"EasyOCR Error on {image_path}: {e}")
        return ""
    

#############################           PaddleOCR            ############################

# import os

# # Avoid slow "model hoster connectivity" checks on startup
# os.environ.setdefault("DISABLE_MODEL_SOURCE_CHECK", "True")

# from paddleocr import PaddleOCR

# # PaddleOCR v3.x: `use_gpu` and `use_angle_cls` are not valid args.
# # For orientation/angle handling, use `use_textline_orientation`.
# # Bump text-det resolution and relax thresholds so small words aren't missed.
# ocr = PaddleOCR(
#     lang='ar',
#     use_textline_orientation=True,
#     text_det_limit_side_len=1280,
#     text_det_limit_type='max',
#     text_det_thresh=0.2,
#     text_det_box_thresh=0.3,
#     text_det_unclip_ratio=1.8,
# )


# def _poly_center(poly):
#     # poly is expected to be (4,2) array-like
#     try:
#         xs = [float(p[0]) for p in poly]
#         ys = [float(p[1]) for p in poly]
#         return (sum(xs) / len(xs), sum(ys) / len(ys))
#     except Exception:
#         return (0.0, 0.0)


# def _group_into_lines(items, y_tol=18.0):
#     """Group (cx, cy, text) into lines by y coordinate."""
#     items = sorted(items, key=lambda t: t[1])
#     lines = []
#     current = []
#     current_y = None

#     for cx, cy, text in items:
#         if current_y is None:
#             current_y = cy
#             current = [(cx, cy, text)]
#             continue

#         if abs(cy - current_y) <= y_tol:
#             current.append((cx, cy, text))
#             current_y = (current_y * (len(current) - 1) + cy) / len(current)
#         else:
#             lines.append(current)
#             current = [(cx, cy, text)]
#             current_y = cy

#     if current:
#         lines.append(current)

#     return lines


# def extractname(image_or_path):
#     """Reads Arabic name text from a path or ndarray using PaddleOCR."""
#     try:
#         src = image_or_path
#         if isinstance(src, str):
#             img = cv2.imread(src)
#         else:
#             img = src
#         if img is None:
#             return ""

#         # Pad the crop a bit to avoid clipping the first/last word at the borders.
#         img = cv2.copyMakeBorder(img, 10, 10, 70, 70, cv2.BORDER_CONSTANT, value=(255, 255, 255))

#         result = ocr.ocr(img)

#         tokens = []

#         # PaddleOCR v3.x output shape: list[dict], with keys like `rec_texts`, `rec_polys`.
#         if isinstance(result, list) and result and isinstance(result[0], dict):
#             for page in result:
#                 rec_texts = page.get('rec_texts') or []
#                 rec_polys = page.get('rec_polys') or []

#                 if isinstance(rec_texts, list) and isinstance(rec_polys, list) and len(rec_texts) == len(rec_polys):
#                     items = []
#                     for t, poly in zip(rec_texts, rec_polys):
#                         if not isinstance(t, str):
#                             continue
#                         t = t.strip()
#                         if not t:
#                             continue
#                         cx, cy = _poly_center(poly)
#                         items.append((cx, cy, t))

#                     # Group into lines by Y, then sort each line RTL (X desc)
#                     for line in _group_into_lines(items):
#                         line_sorted = sorted(line, key=lambda t: t[0], reverse=True)
#                         tokens.extend([t for _, __, t in line_sorted])
#                 else:
#                     # Fallback: just take the text list order as-is
#                     if isinstance(rec_texts, list):
#                         tokens.extend([t.strip() for t in rec_texts if isinstance(t, str) and t.strip()])
#                     elif isinstance(rec_texts, str) and rec_texts.strip():
#                         tokens.append(rec_texts.strip())

#         else:
#             # Older output shape: [[ [box, (text, score)], ... ]]
#             try:
#                 for line in result:
#                     for res in line:
#                         t = res[1][0]
#                         if isinstance(t, str) and t.strip():
#                             tokens.append(t.strip())
#             except Exception:
#                 pass

#         # Clean: keep Arabic letters and spaces
#         if tokens:
#             joined = " ".join(tokens)
#             joined = re.sub(r'[^\u0621-\u064A\s]', '', joined)
#             joined = re.sub(r'\s+', ' ', joined).strip()
#             return joined

#         return ""

#     except Exception as e:
#         print(f"PaddleOCR Error on {type(image_or_path).__name__}: {e}")
#         return ""

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [28]:
def calculate_pipeline_accuracy(true_file_path, extracted_file_path):
    # 1. Load the Excel files
    try:
        # Load files (Sheet 1 is default, which is what we want now)
        df_true = pd.read_excel(true_file_path)
        df_extracted = pd.read_excel(extracted_file_path)
    except Exception as e:
        print(f"Error loading files: {e}")
        return 0.0

    # --- CRITICAL FIX: Clean Column Names ---
    # This removes leading/trailing spaces from headers (e.g., ' Name' -> 'Name')
    df_true.columns = df_true.columns.str.strip()
    df_extracted.columns = df_extracted.columns.str.strip()

    # 2. Align the data lengths
    min_len = min(len(df_true), len(df_extracted))
    df_true = df_true.iloc[:min_len].reset_index(drop=True)
    df_extracted = df_extracted.iloc[:min_len].reset_index(drop=True)

    # Use the clean names now
    columns_to_check = ['Code', 'Daf3', 'Name']
    scores = {}

    print(f"--- Accuracy Report (Checking {min_len} rows) ---\n")

    for col in columns_to_check:
        # Check if column exists (now robust to spaces)
        if col not in df_true.columns or col not in df_extracted.columns:
            print(f"Error: Column '{col}' missing.")
            print(f"   Available in True File: {df_true.columns.tolist()}")
            print(f"   Available in Extracted: {df_extracted.columns.tolist()}")
            scores[col] = 0.0
            continue

        # --- Data Normalization ---
        true_series = df_true[col].astype(str).fillna('')
        extracted_series = df_extracted[col].astype(str).fillna('')

        true_clean = true_series.str.strip().str.replace(r'\.0$', '', regex=True)
        extracted_clean = extracted_series.str.strip().str.replace(r'\.0$', '', regex=True)

        # --- COMPARISON LOGIC ---
        if col == 'Name':
            row_scores = []
            
            for t_val, e_val in zip(true_clean, extracted_clean):
                
                # --- CHECK 1: STRICT ONE-SPACE TOLERANCE ---
                t_nospace = t_val.replace(" ", "")
                e_nospace = e_val.replace(" ", "")
                
                # Perfect match
                if t_val == e_val:
                    row_scores.append(1.0)
                    
                # Match if characters are same AND length differs by only 1
                elif (t_nospace == e_nospace) and (abs(len(t_val) - len(e_val)) <= 1):
                    row_scores.append(1.0) 
                    
                # --- CHECK 2: Fallback (Partial Word Match) ---
                else:
                    t_words = set(t_val.split())
                    e_words = set(e_val.split())
                    
                    if len(t_words) == 0:
                        row_scores.append(1.0 if len(e_words) == 0 else 0.0)
                    else:
                        common = t_words.intersection(e_words)
                        score = len(common) / len(t_words)
                        row_scores.append(score)
            
            accuracy = np.mean(row_scores) * 100
            
        else:
            # === STRICT COMPARISON FOR CODES ===
            matches = (true_clean == extracted_clean) 
            accuracy = (matches.sum() / len(matches)) * 100

        scores[col] = accuracy
        print(f"{col} Accuracy: {accuracy:.2f}%")

    # 3. Calculate Average
    if scores:
        average_accuracy = sum(scores.values()) / len(scores)
    else:
        average_accuracy = 0.0

    print(f"\n--------------------------------")
    print(f"AVERAGE ACCURACY: {average_accuracy:.2f}%")
    print(f"--------------------------------")

    return average_accuracy

In [29]:
#import easyocr

# Initialize the reader for Arabic and English
# The model weights will download automatically on the first run
#reader = easyocr.Reader(['ar', 'en'], gpu=True) # Set gpu=False if you don't have one

# Perform OCR on an image
#results = reader.readtext('path_to_your_image.jpg')

# Print the recognized text
#for (bbox, text, prob) in results:
    #print(f"Text: {text} (Confidence: {prob:.4f})")

# **Main Pipeline** 

In [30]:
import os
import re
import pandas as pd
import cv2
from IPython.display import display, Image as IPyImage, HTML

def show_image_cv(img, title=None):
    import matplotlib.pyplot as plt
    plt.figure(figsize=(4, 4))
    if len(img.shape) == 2:
        plt.imshow(img, cmap='gray')
    else:
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()


def main_pipeline():
    base_dir = os.getcwd()
    path_to_dataset = os.path.join(base_dir, 'Raw_IDs')
    refrence_image_path = os.path.join(base_dir, 'Raw_IDs', 'ID14.jpg')

    # Ensure the classifier is trained
    SVMclassifier = train_SVM_robust()

    data_for_excel = []

    # Safety check for directory
    if not os.path.exists(path_to_dataset):
        print(f"Directory not found: {path_to_dataset}")
        return

    for i in os.listdir(path_to_dataset):
        if not i.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            continue
        
        img_path = os.path.join(path_to_dataset, i)

        # --- Processing ---
        raw_img = cv2.imread(img_path)
        clean_img, is_impulsive = is_impulsive_noise(raw_img)
        aligned_img = align_images_sift(clean_img, refrence_image_path)
        clean_img2, is_random = is_random_noise(aligned_img)
        enhanced_img = enhance_contrast_clahe(clean_img2)
        name_img, digit_imgs, daf3_digits = extract_name_and_digits(enhanced_img)
        student_id = os.path.splitext(i)[0]

        # Save images
        save_student_name(student_id, name_img)
        save_split_digits(student_id, digit_imgs)
        save_split_digits(f"{student_id}_daf3", daf3_digits, output_folder="extracted_daf3_digits")

        # Predict Code
        digit_preds = []
        for digit_img in digit_imgs:
            feat = extract_hog_features(digit_img)
            pred = SVMclassifier.predict([feat])[0]
            digit_preds.append(str(pred))
        code_str = ''.join(digit_preds)

        # Predict Daf3
        daf3_preds = []
        for d_img in daf3_digits:
            feat = extract_hog_features(d_img)
            pred = SVMclassifier.predict([feat])[0]
            daf3_preds.append(str(pred))
        daf3_str = ''.join(daf3_preds)

        # Extract Name
        name_text = extractname(f'./extracted_names/{student_id}_name.jpg')

        # Add to list
        data_for_excel.append({
            "Student ID": student_id,   
            "Name": name_text,
            "Code": code_str,
            "Daf3": daf3_str,
        })

    # --- OUTPUT SECTION ---
    if data_for_excel:
        df = pd.DataFrame(data_for_excel)
        # Sort by Student ID number if needed
        df = df.sort_values(by="Student ID", key=lambda x: x.str.extract(r'(\d+)').iloc[:, 0].astype(int))

        # 1. DISPLAY TABLE IN JUPYTER (with Student ID) + 3 correctness columns
        print("Processing Complete. Results:")
        true_results_path = 'True_Results.xlsx'

        df_display = df[['Student ID', 'Name', 'Code', 'Daf3']].copy()
        if os.path.exists(true_results_path):
            try:
                true_df = pd.read_excel(true_results_path)
                true_df.columns = true_df.columns.str.strip()

                def _key(c: str) -> str:
                    return re.sub(r"[^a-z0-9]", "", str(c).strip().lower())

                colmap = {_key(c): c for c in true_df.columns}
                sid_col = colmap.get('studentid') or colmap.get('student_id') or colmap.get('id')
                if sid_col is None:
                    raise KeyError(f"No student-id column found. Columns: {list(true_df.columns)}")
                true_df = true_df.rename(columns={sid_col: 'Student ID'})

                name_col = colmap.get('name')
                code_col = colmap.get('code')
                daf3_col = colmap.get('daf3')
                if name_col and name_col != 'Name':
                    true_df = true_df.rename(columns={name_col: 'Name'})
                if code_col and code_col != 'Code':
                    true_df = true_df.rename(columns={code_col: 'Code'})
                if daf3_col and daf3_col != 'Daf3':
                    true_df = true_df.rename(columns={daf3_col: 'Daf3'})

                left = df_display.copy()
                right = true_df.copy()
                left['_sid_key'] = left['Student ID'].astype(str).str.extract(r'(\d+)')[0]
                right['_sid_key'] = right['Student ID'].astype(str).str.extract(r'(\d+)')[0]
                merged = left.merge(right, on='_sid_key', how='left', suffixes=('', '_True'))

                def _norm_series(s: pd.Series) -> pd.Series:
                    return s.astype(str).fillna('').str.strip().str.replace(r'\.0$', '', regex=True)

                merged['Name_Correct'] = _norm_series(merged['Name']) == _norm_series(merged.get('Name_True', ''))
                merged['Code_Correct'] = _norm_series(merged['Code']) == _norm_series(merged.get('Code_True', ''))
                merged['Daf3_Correct'] = _norm_series(merged['Daf3']) == _norm_series(merged.get('Daf3_True', ''))

                df_display = merged[[
                    'Student ID',
                    'Name', 'Name_Correct',
                    'Code', 'Code_Correct',
                    'Daf3', 'Daf3_Correct',
                ]]
            except Exception as e:
                print(f"Could not add correctness columns: {e}")

        display(df_display)

        # 2. SAVE TO EXCEL
        output_file = "Extracted_Results.xlsx"
        with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
            df.to_excel(writer, index=False, sheet_name='Sheet1')

            # Auto-resize columns
            worksheet = writer.sheets['Sheet1']
            for column in df:
                column_length = max(df[column].astype(str).map(len).max(), len(column))
                col_idx = df.columns.get_loc(column)
                col_letter = chr(65 + col_idx)
                worksheet.column_dimensions[col_letter].width = column_length + 2

# ... (Excel saving code) ...

        print(f"Excel file saved to: {output_file}")
        
        # <--- CHANGE 3: The call happens here, after the file exists
        true_results_path = 'True_Results.xlsx' 
        
        if os.path.exists(true_results_path):
            calculate_pipeline_accuracy(true_results_path, output_file)
        else:
            print(f"Skipping accuracy check: '{true_results_path}' not found.")

# Run it
main_pipeline()

Loading 149 training images...
Training Complete. Validation Accuracy: 100.00%


C:\Users\ahmed\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\ahmed\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\ahmed\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\ahmed\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\ahmed\AppData\R

Processing Complete. Results:


,Student ID,Name,Name_Correct,Code,Code_Correct,Daf3,Daf3_Correct
0,ID1,مصطفى محمود احمد عويس,True,1200277,True,14712020100041,True
1,ID2,شاميه علاء محمد سيد احمد,True,1220175,True,14712022101,False
2,ID3,اسمهان ابراهيم يوسف البيطار,True,1220165,True,14712022101214,True
3,ID4,حلمي علي ريان,False,1220145,True,14712022101495,True
4,ID5,حسين محمد ماهر بهاءالدين,False,1220237,True,14712022101358,True
5,ID6,محمد منير عبدالحميد كمال,True,4230174,True,14712022101524,True
6,ID7,يوسف محمد حمدى محمل عثمان,False,1220301,True,14712522101392,False
7,ID8,معاذ امام احمد امام احمد,True,1220205,True,14712022101329,True
8,ID9,نغم طارق مجمد احمد قاسم,False,1220149,True,14712022101620,True
9,ID10,يوسف عمرو سيد كمال محمود محمد,True,1220319,True,14712022101322,True


Excel file saved to: Extracted_Results.xlsx
--- Accuracy Report (Checking 54 rows) ---

Code Accuracy: 100.00%
Daf3 Accuracy: 92.59%
Name Accuracy: 91.60%

--------------------------------
AVERAGE ACCURACY: 94.73%
--------------------------------


In [31]:
# # Quick sanity check for PaddleOCR runtime
# import os

# os.environ.setdefault("DISABLE_MODEL_SOURCE_CHECK", "True")

# import paddle
# from paddleocr import PaddleOCR

# print('paddle:', paddle.__version__)
# print('cuda compiled:', paddle.device.is_compiled_with_cuda())

# # PaddleOCR v3.x: no `use_gpu`; CPU is used automatically with CPU paddle builds
# ocr = PaddleOCR(lang='ar', use_textline_orientation=True)
# print('PaddleOCR ready')

In [32]:
# # Debug: inspect one name crop + PaddleOCR detected tokens
# import os
# import cv2
# import numpy as np

# def _poly_center(poly):
#     try:
#         xs = [float(p[0]) for p in poly]
#         ys = [float(p[1]) for p in poly]
#         return (sum(xs) / len(xs), sum(ys) / len(ys))
#     except Exception:
#         return (0.0, 0.0)

# name_dir = 'extracted_names'
# name_files = sorted([f for f in os.listdir(name_dir) if f.lower().endswith(('.jpg','.png','.jpeg'))])
# print('name images:', len(name_files))

# if not name_files:
#     raise RuntimeError(f'No images found in {name_dir!r}')

# sample_file = name_files[0]
# sample_path = os.path.join(name_dir, sample_file)
# img0 = cv2.imread(sample_path)
# print('sample:', sample_file, 'shape:', None if img0 is None else img0.shape, 'mean:', None if img0 is None else float(np.mean(img0)))

# show_image_cv(img0, title=f'Name crop: {sample_file}')

# # Run OCR the same way as extractname()
# img = cv2.copyMakeBorder(img0, 10, 10, 60, 60, cv2.BORDER_CONSTANT, value=(255, 255, 255))
# raw = ocr.ocr(img)

# if isinstance(raw, list) and raw and isinstance(raw[0], dict):
#     page = raw[0]
#     rec_texts = page.get('rec_texts') or []
#     rec_polys = page.get('rec_polys') or []

#     print('\nrec_texts:', rec_texts)

#     if isinstance(rec_texts, list) and isinstance(rec_polys, list) and len(rec_texts) == len(rec_polys):
#         items = []
#         for t, poly in zip(rec_texts, rec_polys):
#             cx, cy = _poly_center(poly)
#             items.append((cx, cy, t))

#         print('\nDetected (sorted RTL by x):')
#         for cx, cy, t in sorted(items, key=lambda x: x[0], reverse=True):
#             print(f'  x={cx:7.1f} y={cy:6.1f}  {t}')

# print('\nextractname():', extractname(sample_path))

## Qwen2.5-VL pretrained OCR (comparison only)
- This section does **not** change your pipeline or existing OCR code.
- It runs a pretrained Qwen2.5-VL OCR model on the already-saved name crops in `extracted_names/` and writes a separate Excel file for comparison.

In [33]:
# # Qwen comparison dependencies (SAFE CHECK-ONLY cell)
# # This cell does NOT install/upgrade anything (no downloads, no env overwrites).
# # If something is missing, it prints the exact one-time commands to run in a Terminal.

# import os, sys, importlib
# from importlib.util import find_spec

# os.environ.setdefault('KMP_DUPLICATE_LIB_OK', 'TRUE')

# import torch
# print('torch:', torch.__version__, 'cuda_available:', torch.cuda.is_available())
# if not torch.cuda.is_available():
#     raise RuntimeError('CUDA is not available in this kernel. Fix the conda CUDA torch stack and restart the kernel.')
# print('GPU:', torch.cuda.get_device_name(0))

# # Versions pinned to avoid pip/conda conflicts (and to keep torch CUDA intact).
# REQUIRED = {
#     'huggingface_hub': '0.34.4',
#     'transformers': '4.57.3',
#     'accelerate': '0.30.1',
# }

# def _get_version(mod_name: str) -> str | None:
#     try:
#         m = importlib.import_module(mod_name)
#         return getattr(m, '__version__', None)
#     except Exception:
#         return None

# problems = []

# # Required core deps (version-pinned)
# for pkg, want in REQUIRED.items():
#     got = _get_version(pkg)
#     if got is None:
#         problems.append(f'{pkg} is missing')
#     elif got != want:
#         problems.append(f'{pkg} is {got} (expected {want})')
#     else:
#         print(f'{pkg}: {got}')

# # Non-version-pinned deps
# for pkg in ('sentencepiece', 'einops'):
#     if find_spec(pkg) is None:
#         problems.append(f'{pkg} is missing')
#     else:
#         v = _get_version(pkg)
#         print(f'{pkg}: {v or "OK"}')

# # bitsandbytes: DO NOT import here (it registers torch ops at import-time).
# if find_spec('bitsandbytes') is None:
#     problems.append('bitsandbytes is missing')
# else:
#     print('bitsandbytes: installed (will be imported in the model cell)')

# if problems:
#     print('\nDependency problems detected:')
#     for p in problems:
#         print(' -', p)
#     print('\nFix once in a Terminal (NOT in a notebook cell):')
#     print('  1) Make sure pip did not install torch/numpy:')
#     print('     python -m pip uninstall -y torch torchvision torchaudio numpy')
#     print('  2) Restore CUDA torch via conda (if needed):')
#     print('     conda install -y -n imageproc pytorch=2.2.2 torchvision=0.17.2 torchaudio=2.2.2 pytorch-cuda=12.1 -c pytorch -c nvidia')
#     print('  3) Install the pinned Qwen deps WITHOUT deps (so pip cannot overwrite torch):')
#     print('     python -m pip install --no-deps huggingface-hub==0.34.4 transformers==4.57.3 accelerate==0.30.1 bitsandbytes==0.47.0 sentencepiece einops')
#     print('  4) Restart the notebook kernel, then re-run this cell.')
#     raise RuntimeError('Qwen deps are not in the expected safe state. See printed fix commands.')

# print('\nQwen deps look safe. Next: run the model-load cell (it may download model weights to the HF cache).')

In [34]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor
import torch

# Transformers may call torch.compiler.is_compiling() (Torch 2.3+).
# Torch 2.2 provides torch.compiler but can miss is_compiling, so we shim it.
try:
    import torch.compiler as _torch_compiler
    if not hasattr(_torch_compiler, "is_compiling"):
        if hasattr(torch, "_dynamo") and hasattr(torch._dynamo, "is_compiling"):
            _torch_compiler.is_compiling = torch._dynamo.is_compiling
        else:
            _torch_compiler.is_compiling = lambda: False
except Exception:
    pass

from PIL import Image
from typing import List
import os
import sys
import re
import pandas as pd
import numpy as np
from IPython.display import display, Image as IPyImage, HTML, clear_output

os.environ.setdefault('KMP_DUPLICATE_LIB_OK', 'TRUE')

# -----------------------
# Notebook progress knobs
# -----------------------
SHOW_LIVE_PROGRESS = True          # prints status and updates live
SHOW_IMAGE_PREVIEW = True          # show the current crop image inline
SHOW_EVERY_N = 1                   # show every N images (1 = all)
MAX_PREVIEW = 30                   # cap previews to avoid flooding the notebook
SHOW_TABLE_EVERY_N = 10            # update a small results table every N images


def _maybe_display_image(path: str, width: int = 500):
    if not SHOW_IMAGE_PREVIEW:
        return
    if not os.path.exists(path):
        return
    try:
        display(HTML(f"<div style='margin:6px 0; font-weight:600'>Preview: {os.path.basename(path)}</div>"))
        display(IPyImage(filename=path, width=width))
    except Exception:
        # If display fails for any reason, just skip preview.
        pass


if not torch.cuda.is_available():
    raise RuntimeError('CUDA is not available. Fix torch CUDA first, then restart kernel and re-run.')
print('Using GPU:', torch.cuda.get_device_name(0))

# bitsandbytes can already be imported in the kernel (e.g., from a previous run).
# That's okay; we warn instead of stopping.
if 'bitsandbytes' in sys.modules:
    print('Note: bitsandbytes is already imported in this kernel; continuing.')


def process_vision_info(messages: List[dict]):
    image_inputs = []
    video_inputs = []
    for message in messages:
        if isinstance(message["content"], list):
            for item in message["content"]:
                if item["type"] == "image":
                    image = item["image"]
                    if isinstance(image, str):
                        image = Image.open(image).convert("RGB")
                    elif isinstance(image, Image.Image):
                        pass
                    else:
                        raise ValueError(f"Unsupported image type: {type(image)}")
                    image_inputs.append(image)
                elif item["type"] == "video":
                    video_inputs.append(item["video"])
    return image_inputs if image_inputs else None, video_inputs if video_inputs else None


def _clean_arabic_name(text: str) -> str:
    if not text:
        return ''
    # Keep Arabic letters + spaces only (removes digits/codes/latin/punctuation).
    cleaned = re.sub(r"[^\u0621-\u064A\s]", " ", str(text))
    cleaned = re.sub(r"\s+", " ", cleaned).strip()
    return cleaned


def _normalize_name_value(v) -> str:
    s = '' if v is None else str(v)
    s = s.strip()
    s = s.replace('\u200f', '').replace('\u200e', '')
    s = s.replace('\xa0', ' ')
    s = s.replace('\t', ' ')
    s = s.replace('\r', ' ').replace('\n', ' ')
    s = re.sub(r"\s+", " ", s).strip()
    if s.endswith('.0'):
        s = s[:-2]
    return s.strip()


def _exact_name_match(true_name: str, pred_name: str) -> bool:
    return _normalize_name_value(true_name) == _normalize_name_value(pred_name)


def _is_name_correct(true_name: str, pred_name: str) -> bool:
    # Your existing tolerance rule (used for accuracy scoring): exact match OR one-space tolerance
    t = _normalize_name_value(true_name)
    p = _normalize_name_value(pred_name)
    if t == p:
        return True
    t_nospace = t.replace(' ', '')
    p_nospace = p.replace(' ', '')
    return (t_nospace == p_nospace) and (abs(len(t) - len(p)) <= 1)


def _name_score(true_name: str, pred_name: str) -> float:
    t = _normalize_name_value(true_name)
    p = _normalize_name_value(pred_name)
    # Perfect match
    if t == p:
        return 1.0
    # One-space tolerance
    if _is_name_correct(t, p):
        return 1.0
    # Partial word match
    t_words = set(t.split())
    p_words = set(p.split())
    if len(t_words) == 0:
        return 1.0 if len(p_words) == 0 else 0.0
    common = t_words.intersection(p_words)
    return len(common) / len(t_words)


def _name_accuracy(true_names: List[str], pred_names: List[str]) -> float:
    scores = [_name_score(t, p) for t, p in zip(true_names, pred_names)]
    return float(np.mean(scores) * 100) if scores else 0.0


# 4-bit model (requires bitsandbytes).
model_name = "sherif1313/Arabic-handwritten-OCR-4bit-Qwen2.5-VL-3B-v2"
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_name,
    device_map="auto",
    dtype=torch.float16,
    trust_remote_code=True,
    low_cpu_mem_usage=True,
)

try:
    device_map = getattr(model, 'hf_device_map', None)
    if isinstance(device_map, dict):
        print('hf_device_map:', device_map)
except Exception:
    pass

processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)


def extract_name_from_image(image_path: str) -> str:
    # Ask for the student's name only (no codes / numbers).
    prompt = (
        "استخرج اسم الطالب فقط من الصورة. "
        "اكتب الاسم العربي فقط بدون أرقام أو رموز أو كلمات إضافية."
    )
    try:
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image_path},
                    {"type": "text", "text": prompt},
                ],
            }
        ]

        text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        image_inputs, _video_inputs = process_vision_info(messages)
        inputs = processor(
            text=[text],
            images=image_inputs,
            padding=True,
            return_tensors="pt",
        ).to(model.device)

        generated_ids = model.generate(
            **inputs,
            max_new_tokens=128,
            do_sample=False,
            pad_token_id=processor.tokenizer.eos_token_id,
            eos_token_id=processor.tokenizer.eos_token_id,
        )

        input_len = inputs.input_ids.shape[1]
        output_text = processor.batch_decode(
            generated_ids[:, input_len:],
            skip_special_tokens=True,
            clean_up_tokenization_spaces=True,
        )[0]
        return _clean_arabic_name(output_text)
    except Exception as e:
        print(f"Error on {os.path.basename(image_path)}: {e}")
        return ''


###############################################################################
# Run Qwen on name crops ONLY and evaluate ONLY name accuracy.
###############################################################################

NAMES_DIR = 'extracted_names'
BASELINE_XLSX = 'Extracted_Results.xlsx'
TRUE_XLSX = 'True_Results.xlsx'
QWEN_XLSX = 'Qwen_Results.xlsx'

if not os.path.isdir(NAMES_DIR):
    raise RuntimeError(f"{NAMES_DIR!r} folder not found. Run the main pipeline first to generate name crops.")
if not os.path.exists(BASELINE_XLSX):
    raise RuntimeError(f"{BASELINE_XLSX!r} not found. Run the main pipeline first (it generates this file).")

df_base = pd.read_excel(BASELINE_XLSX)
df_base.columns = df_base.columns.str.strip()
if 'Student ID' not in df_base.columns:
    raise RuntimeError(f"{BASELINE_XLSX!r} is missing 'Student ID' column.")

# Run Qwen for each student's name crop
qwen_rows = []
preview_count = 0
n_total = int(df_base.shape[0])

for i, sid in enumerate(df_base['Student ID'].astype(str).tolist(), start=1):
    img_path = os.path.join(NAMES_DIR, f"{sid}_name.jpg")
    if not os.path.exists(img_path):
        alt = os.path.join(NAMES_DIR, f"{sid}.jpg")
        img_path = alt if os.path.exists(alt) else img_path

    do_show = (SHOW_LIVE_PROGRESS and (i % int(SHOW_EVERY_N) == 0))

    if do_show:
        clear_output(wait=True)
        print(f"Qwen processing: {i}/{n_total} | Student ID: {sid}")
        print(f"Image: {img_path}")

        if preview_count < int(MAX_PREVIEW):
            _maybe_display_image(img_path)
            preview_count += 1

    qwen_name = extract_name_from_image(img_path) if os.path.exists(img_path) else ''
    qwen_rows.append({'Student ID': sid, 'Name': qwen_name})

    if do_show:
        print("\nQwen output (cleaned):")
        print(qwen_name)

        if SHOW_TABLE_EVERY_N and (i % int(SHOW_TABLE_EVERY_N) == 0):
            df_live = pd.DataFrame(qwen_rows)
            display(df_live.tail(min(10, len(df_live))))

# Final save
clear_output(wait=False)
df_qwen = pd.DataFrame(qwen_rows)
df_qwen.to_excel(QWEN_XLSX, index=False)
print(f"Saved Qwen name-only results to: {QWEN_XLSX}")
display(df_qwen.head(10))

# Evaluate name accuracy and show comparison table
if not os.path.exists(TRUE_XLSX):
    print(f"Note: {TRUE_XLSX!r} not found, so accuracy/table are skipped.")
    display(df_qwen)
else:
    df_true = pd.read_excel(TRUE_XLSX)
    df_true.columns = df_true.columns.str.strip()

    # Build comparison table aligned by Student ID when possible
    if 'Student ID' in df_true.columns and 'Name' in df_true.columns and 'Name' in df_base.columns:
        df_true_k = df_true[['Student ID', 'Name']].copy()
        df_true_k['Student ID'] = df_true_k['Student ID'].astype(str)
        df_base_k = df_base[['Student ID', 'Name']].copy()
        df_base_k['Student ID'] = df_base_k['Student ID'].astype(str)
        df_qwen_k = df_qwen[['Student ID', 'Name']].copy()
        df_qwen_k['Student ID'] = df_qwen_k['Student ID'].astype(str)

        table = df_true_k.merge(df_base_k, on='Student ID', how='inner', suffixes=('_Actual', '_MyCode'))
        table = table.merge(df_qwen_k, on='Student ID', how='inner')
        table = table.rename(columns={'Name': 'Qwen Output'})
        table = table.rename(columns={'Name_Actual': 'Actual Name', 'Name_MyCode': 'My Code Output'})

        # Right/Wrong uses the tolerance rule (same as accuracy scoring).
        table['Qwen Right/Wrong'] = [
            'Right' if _is_name_correct(a, q) else 'Wrong'
            for a, q in zip(table['Actual Name'], table['Qwen Output'])
        ]

        # Requested boolean columns: True only when extracted == true (exact match after normalization).
        table['My Code == True'] = [
            _exact_name_match(a, m) for a, m in zip(table['Actual Name'], table['My Code Output'])
        ]
        table['Qwen == True'] = [
            _exact_name_match(a, q) for a, q in zip(table['Actual Name'], table['Qwen Output'])
        ]
        table['Both == True'] = table['My Code == True'] & table['Qwen == True']

        baseline_name_acc = _name_accuracy(table['Actual Name'].tolist(), table['My Code Output'].tolist())
        qwen_name_acc = _name_accuracy(table['Actual Name'].tolist(), table['Qwen Output'].tolist())
        print(f"Baseline Name Accuracy: {baseline_name_acc:.2f}%")
        print(f"Qwen Name Accuracy:     {qwen_name_acc:.2f}%")
        print(f"Name Delta (baseline - Qwen): {baseline_name_acc - qwen_name_acc:+.2f} points")

        display(table[['Student ID', 'Actual Name', 'My Code Output', 'Qwen Output', 'Qwen Right/Wrong', 'My Code == True', 'Qwen == True', 'Both == True']])
    else:
        # Fallback: align by row order (if Student ID isn't available in the true file).
        min_len = min(len(df_true), len(df_base), len(df_qwen))
        actual = df_true.get('Name', pd.Series([''] * min_len)).iloc[:min_len].tolist()
        mycode = df_base.get('Name', pd.Series([''] * min_len)).iloc[:min_len].tolist()
        qwen = df_qwen.get('Name', pd.Series([''] * min_len)).iloc[:min_len].tolist()

        baseline_name_acc = _name_accuracy(actual, mycode)
        qwen_name_acc = _name_accuracy(actual, qwen)
        print(f"Baseline Name Accuracy: {baseline_name_acc:.2f}%")
        print(f"Qwen Name Accuracy:     {qwen_name_acc:.2f}%")
        print(f"Name Delta (baseline - Qwen): {baseline_name_acc - qwen_name_acc:+.2f} points")

        table = pd.DataFrame({
            'Row': list(range(1, min_len + 1)),
            'Actual Name': actual,
            'My Code Output': mycode,
            'Qwen Output': qwen,
        })
        table['Qwen Right/Wrong'] = [
            'Right' if _is_name_correct(a, q) else 'Wrong'
            for a, q in zip(table['Actual Name'], table['Qwen Output'])
        ]
        table['My Code == True'] = [
            _exact_name_match(a, m) for a, m in zip(table['Actual Name'], table['My Code Output'])
        ]
        table['Qwen == True'] = [
            _exact_name_match(a, q) for a, q in zip(table['Actual Name'], table['Qwen Output'])
        ]
        table['Both == True'] = table['My Code == True'] & table['Qwen == True']

        display(table[['Row', 'Actual Name', 'My Code Output', 'Qwen Output', 'Qwen Right/Wrong', 'My Code == True', 'Qwen == True', 'Both == True']])


ModuleNotFoundError: No module named 'transformers'